# Moss + LangChain Integration

This notebook demonstrates how to integrate **Moss** (semantic search) with **LangChain** for RAG and Agentic workflows.

## 1. Setup

First, we'll install dependencies and load environment variables.

In [ ]:
%pip install -qU inferedge-moss langchain langchain-openai python-dotenv

In [33]:
import os
from dotenv import load_dotenv
from moss_langchain import MossRetriever, get_moss_tool

# Load environment variables from .env file
load_dotenv()

PROJECT_ID = os.getenv("MOSS_PROJECT_ID")
PROJECT_KEY = os.getenv("MOSS_PROJECT_KEY")
INDEX_NAME = os.getenv("MOSS_INDEX_NAME")

## 2. Using MossRetriever

The `MossRetriever` allows you to fetch relevant documents directly from your Moss index. 

> [!IMPORTANT]
> In Jupyter notebooks, you should use **`ainvoke`** (async) because `invoke` (sync) will conflict with the notebook's running event loop.

In [35]:
retriever = MossRetriever(
    project_id=PROJECT_ID,
    project_key=PROJECT_KEY,
    index_name=INDEX_NAME,
    top_k=3,
    alpha=0.2
)

query = "Whats the return policy"
docs = await retriever.ainvoke(query)

for i, doc in enumerate(docs):
    print(f"--- Result {i+1} (Score: {doc.metadata['score']:.4f}) ---")
    print(doc.page_content[:200] + "...")
    print()

--- Result 1 (Score: 0.9937) ---
What is your return policy? We offer a 30-day return policy for most items....

--- Result 2 (Score: 0.9394) ---
What is your return policy? We offer a 34-day return policy for most items....

--- Result 3 (Score: 0.7750) ---
What is your return policy? We offer a 34-day return policy for most items....



## 3. Retrieval-Augmented Generation (RAG)

Now we'll use the retriever in a simple LangChain chain to answer questions based on the retrieved context.

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4.1-mini",api_key=os.getenv(OPENAI_API_KEY))

prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the provided context:

<context>
{context}
</context>

Question: {question}
""")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = await rag_chain.ainvoke("What is the shipping address?")
print(response)

The shipping address is near St. Mark's Square.


## 4. Agent Tool

For more complex workflows, you can expose Moss as a tool for an agent to use autonomously.

In [36]:
from langchain_classic.agents import AgentExecutor, create_openai_functions_agent
from langchain_classic import hub

# Get the pre-configured Moss tool
moss_tool = get_moss_tool(PROJECT_ID, PROJECT_KEY, INDEX_NAME)

tools = [moss_tool]

# Get the prompt to use - can be modified
prompt = hub.pull("hwchase17/openai-functions-agent")

# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

await agent_executor.ainvoke({"input": "Can you check the refund policy and whats the return policy for international orders?"})




> Entering new AgentExecutor chain...

Invoking: `moss_search` with `refund policy`


Result 1:
What is your return policy? We offer a 30-day return policy for most items.

Result 2:
What is your return policy? We offer a 34-day return policy for most items.

Result 3:
What is your return policy? We offer a 34-day return policy for most items.

Result 4:
What is your return policy? We offer a 34-day return policy for most items.

Result 5:
What is your return policy? We offer a 34-day return policy for most items.
Invoking: `moss_search` with `return policy for international orders`


Result 1:
What is your return policy for international orders? International returns are accepted within 45 days. Return shipping costs are the responsibility of the customer unless the item is defective.

Result 2:
What is your return policy? We offer a 30-day return policy for most items.

Result 3:
What is your return policy? We offer a 34-day return policy for most items.

Result 4:
What is your ret

{'input': 'Can you check the refund policy and whats the return policy for international orders?',
 'output': 'The general return policy offers a 34-day return period for most items. \n\nFor international orders specifically, returns are accepted within 45 days. However, the return shipping costs for international returns are the responsibility of the customer unless the item is defective.\n\nIf you need information about the refund policy specifically, please let me know!'}